<a href="https://colab.research.google.com/github/eikku/JODA-guest-lecture/blob/main/yhteenveto_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip3 install --upgrade --user google-cloud-aiplatform

In [5]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import pandas as pd
#lataa datasetti, lataa omaan käyttöön https://digi.kansalliskirjasto.fi/opendata
url = 'https://raw.githubusercontent.com/eikku/JODA-guest-lecture/main/digi_collectioninfo_202311116.csv' #raw content
df = pd.read_csv(url, sep=";",lineterminator='\n',encoding='utf-8' )

In [2]:
df.head()

,ID,PARENT_ID,NIMI,BANNER_URL,KUVAUS_FI,KUVAUS_SV,KUVAUS_EN,NIMI_EN,NIMI_SV,CollectionPAGELink\r
0,101,NaN,Yhteistyössä toteutetut,images/collections/sancol.png,Valittuja uudempia aineistoja digitoidaan ja a...,"Utvalda, ofta nyare material digitaliseras och...","Selected, usually newer materials are digitise...",Selections made in collaboration,Verk som digitaliserats med stöd från samarbet...,https://digi.kansalliskirjasto.fi/collections?...
1,102,101.0,Käkisalmen ja Laatokan alueen lehdet,images/collections/laatokka_musketti.png,Vanhoja Käkisalmen ja Laatokan alueen sanomale...,De historiska tidningarna och tidskrifterna fr...,Priozersk and Lake Ladoga region newspapers an...,Priozersk and Lake Ladoga region newspapers an...,Tidningar och tidskrifter från Kexholm och Ladoga,https://digi.kansalliskirjasto.fi/collections?...
2,201,501.0,Maantiede ja matkailu,/images/collections/kartat.png,Valikoima Suomea koskevaa tai suomalaisten kir...,Ett urval litteratur om geografi och turism fr...,This is a selection of geography and travel li...,Geography and travel,Geografi och resor,https://digi.kansalliskirjasto.fi/collections?...
3,441,101.0,Bibliotheca Wiburgensis,/images/collections/kaki.jpg,Kokoelma sisältää valikoiman Viipurin Suomalai...,Samlingen innehåller ett urval böcker som publ...,The collection includes a selection of books p...,Bibliotheca Wiburgensis,Bibliotheca Wiburgensis,https://digi.kansalliskirjasto.fi/collections?...
4,501,NaN,Kirjat,images/collections/colbanner6.jpg,NaN,NaN,NaN,Books,Böcker,https://digi.kansalliskirjasto.fi/collections?...


In [3]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [13]:
from google.colab import userdata
# Define project information
PROJECT_ID = userdata.get('PROJECT_ID')  # @param {type:"string"}
LOCATION = "europe-west4"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [14]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
    GenerationConfig,
)

In [15]:
model = GenerativeModel("gemini-1.0-pro")

In [16]:
##Määritellään mallin parametrejä
generation_config = GenerationConfig(
    temperature=0.5,
    top_p=1.0,
    top_k=32,
    candidate_count=1,
    max_output_tokens=8192,
)

In [17]:
KUVAUS=df.KUVAUS_FI.tolist()

In [24]:
##PROMT ESIMERKKI
prompt = f"""Luo lyhyt yhteenveto seuraavista aineistoista ja luokittele ne kolmeen luokkaan :

{KUVAUS}
"""

print(prompt)

Luo lyhyt yhteenveto seuraavista aineistoista ja luokittele ne kolmeen luokkaan : 

['Valittuja uudempia aineistoja digitoidaan ja avataan yhteistyökumppaneiden tukemana.', 'Vanhoja Käkisalmen ja Laatokan alueen sanomalehtiä pääsee nyt lukemaan digi.kansalliskirjasto.fi -verkkopalvelussa. <a target="_new" href="https://kakisalmi.fi/">Käkisalmi-Säätiö</a>, Kansalliskirjasto ja Kopiosto ovat sopineet lehtien digitoinnista ja avoimesta verkkokäytöstä.\n\n<a target="_new" href="https://www.kansalliskirjasto.fi/fi/uutiset/historialliset-karjalaiset-sanomalehdet-digikayttoon">Tiedote Kansalliskirjaston sivuilla</a>\n\nKokoelman kuva: "Laatokka kesäyönä" - <a target="_new" href="https://www.finna.fi/Record/musketti.M012:HK19700502:197">Museovirasto - Musketti</A> . \nKokoelman kuvituskuva on julkaistu Nimeä 4.0 Kansainvälinen (CC BY 4.0)-lisenssillä.   ![CC 4.0 BY](images/licenses/by_88x31.png) (Kuvan käyttöoikeudet tarkastettu 21.11.2018)', 'Valikoima Suomea koskevaa tai suomalaisten kirjoit

In [25]:
responses = model.generate_content(prompt,  generation_config=generation_config, stream=True)

for response in responses:
    print(response.text, end="")

**Digitoitu aineisto**

* Vanhoja Käkisalmen ja Laatokan alueen sanomalehtiä
* Valikoima Suomea koskevaa tai suomalaisten kirjoittamaa maantiedettä ja matkailua käsittelevää kirjallisuutta
* Viipurin Suomalaisen Kirjallisuusseuran julkaisemia kirjoja ja viipurilaisten kirjoittamia kauno- ja tietokirjallisuusteoksia
* Suomen Sosialidemokraatti, Työkansan Sanomat, Vapaa sana ja Kansan Uutiset -lehtien numerot
* Academica-kokoelma vuoteen 1827
* Suomen-, ruotsin- ja latinankieliset asetukset
* Ruotsinkieliset kaupunginkalenterit, tiedeakatemian kalenterit ja hovikalenterit
* Suomen-, ruotsin- ja latinankielistä kaunokirjallisuutta
* Suomen- ja ruotsinkieliset almanakat
* Hengellisten laulukirjojen lisäksi jonkin verran arkkiveisuja
* Ruumissaarnoja, hautajaispuheita ja onnittelurunoja
* Historia-alakokoelma
* Kirkollisia käsikirjoja ja evankeliumikirjoja
* Sotaväen asetuksia, ohjesääntöjä, sotalakeja ja erillisteoksia
* Saarnakirjallisuuden kokoelma
* Tuomiokapitulien kiertokirjeitä, kirk

In [26]:
##nätimpi tulostus
from IPython.display import display, Markdown


### Puetaan funktioon
Käsitellään suoraan annettua csv

In [34]:
instructions = """Olet kvalitatiivisen analyysin asiantuntija.
Luo vastauksista yhteenveto, jossa luokittelet vastauksia eri kategorioihin:"""

# Function to iterate over DataFrame rows and generate summaries
def generate_feedback_summaries(df,column):
    # Apply the model's generate_content method to each feedback entry
    df_mod=df.dropna(subset=[column])
    df_mod['summary'] = df_mod[column].apply(lambda x: model.generate_content(instructions+x,generation_config=generation_config).text)
    return df_mod

# Apply the function to your DataFrame
df_with_summaries = generate_feedback_summaries(df.head(),'KUVAUS_FI')

<ipython-input-34-692e88ce498d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mod['summary'] = df_mod[column].apply(lambda x: model.generate_content(instructions+x,generation_config=generation_config).text)


In [35]:
df_with_summaries.head()

,ID,PARENT_ID,NIMI,BANNER_URL,KUVAUS_FI,KUVAUS_SV,KUVAUS_EN,NIMI_EN,NIMI_SV,CollectionPAGELink\r,summary
0,101,NaN,Yhteistyössä toteutetut,images/collections/sancol.png,Valittuja uudempia aineistoja digitoidaan ja a...,"Utvalda, ofta nyare material digitaliseras och...","Selected, usually newer materials are digitise...",Selections made in collaboration,Verk som digitaliserats med stöd från samarbet...,https://digi.kansalliskirjasto.fi/collections?...,**Digitointi ja avoimuus**\n\n* Valittuja uude...
1,102,101.0,Käkisalmen ja Laatokan alueen lehdet,images/collections/laatokka_musketti.png,Vanhoja Käkisalmen ja Laatokan alueen sanomale...,De historiska tidningarna och tidskrifterna fr...,Priozersk and Lake Ladoga region newspapers an...,Priozersk and Lake Ladoga region newspapers an...,Tidningar och tidskrifter från Kexholm och Ladoga,https://digi.kansalliskirjasto.fi/collections?...,**Kategorioita ei ole määritelty vastauksissa.**
2,201,501.0,Maantiede ja matkailu,/images/collections/kartat.png,Valikoima Suomea koskevaa tai suomalaisten kir...,Ett urval litteratur om geografi och turism fr...,This is a selection of geography and travel li...,Geography and travel,Geografi och resor,https://digi.kansalliskirjasto.fi/collections?...,**Kategorioita:**\n\n**Maantieteen oppikirjat*...
3,441,101.0,Bibliotheca Wiburgensis,/images/collections/kaki.jpg,Kokoelma sisältää valikoiman Viipurin Suomalai...,Samlingen innehåller ett urval böcker som publ...,The collection includes a selection of books p...,Bibliotheca Wiburgensis,Bibliotheca Wiburgensis,https://digi.kansalliskirjasto.fi/collections?...,**Kokoelman sisällön luokittelut:**\n\n* **Vii...


In [36]:
df_with_summaries[['KUVAUS_FI','summary']]

,KUVAUS_FI,summary
0,Valittuja uudempia aineistoja digitoidaan ja a...,**Digitointi ja avoimuus**\n\n* Valittuja uude...
1,Vanhoja Käkisalmen ja Laatokan alueen sanomale...,**Kategorioita ei ole määritelty vastauksissa.**
2,Valikoima Suomea koskevaa tai suomalaisten kir...,**Kategorioita:**\n\n**Maantieteen oppikirjat*...
3,Kokoelma sisältää valikoiman Viipurin Suomalai...,**Kokoelman sisällön luokittelut:**\n\n* **Vii...


In [43]:
# Display Name and Markdown Description
for _, row in df_with_summaries.iterrows():
    # Display the Name with some formatting (e.g., as a heading)
    display(Markdown(f"### {row['NIMI']}"))

    # Display the Markdown formatted Description
    display(Markdown(row['summary']))

### Yhteistyössä toteutetut

**Digitointi ja avoimuus**

* Valittuja uudempia aineistoja digitoidaan ja avataan yhteistyökumppaneiden tukemana.

### Käkisalmen ja Laatokan alueen lehdet

**Kategorioita ei ole määritelty vastauksissa.**

### Maantiede ja matkailu

**Kategorioita:**

**Maantieteen oppikirjat**
* Ei mainittu

**Matkaoppaat**
* Venäjänkieliset matkaopaskirjallisuus Suomeen

**Matkakirjat**
* Ulkomaisten matkailijoiden kuvauksia Suomesta
* Suomalaisten matkakertomuksia maailmalta (Palestiina, Brasilia, Afrikka, Siperia)

**Amerikka-kirjallisuus**
* Suomalaisille matkustavaisille tarkoitettua

**Lappiin liittyvä materiaali**
* Ei mainittu

**Muut**
* Ei mainittu

### Bibliotheca Wiburgensis

**Kokoelman sisällön luokittelut:**

* **Viipurin Suomalaisen Kirjallisuusseuran (VSKS) julkaisut:**
    * Kirjat vuosilta 1847-1882

* **Viipurilaisten kirjoittamat teokset:**
    * Kaunokirjallisuus
    * Tietokirjallisuus

* **Cedervallerien Viipurin kirjapainon painamat kirjat:**
    * Kirjat painettu ennen vuotta 1860